In [1]:
import pandas as pd
import numpy as np

# Load the data (replace with your actual file paths or database queries)
usersorder_df = pd.read_excel(r"USER AND RESTRAUNT.xlsx")  # Contains columns: user_id, rest_id, cost, rating, location
restaurants_df = pd.read_csv(r"BangaloreZomatoData_with_rest_id.csv")



In [2]:
usersorder_df.head()

,rest_id,Name,user_id,cost,rating,location,Cuisines
0,R6892,Indian Food Court,U0350,1000,4,"Electronic City, Bangalore","North Indian, Chinese, Fast Food"
1,R7304,Ludos Pizza,U0825,800,2,"JP Nagar, Bangalore","Pizza, Desserts"
2,R1631,Seth Chatpata Chatwala,U0204,500,3,"KR Puram, Bangalore","Fast Food, Pizza, Street Food, Beverages, Shake"
3,R7506,Meghana Foods,U0300,300,2,"Sahakara Nagar, Bangalore","Biryani, Andhra, North Indian, Seafood"
4,R3521,Mealful Rolls - Meals In A Roll,U0479,400,5,"Jayanagar, Bangalore","Rolls, Fast Food"


In [3]:
def get_previous_ratings(user_id, usersorder_df):
    # Filter restaurants rated by the user
    user_ratings = usersorder_df[usersorder_df['user_id'] == user_id]
    
    # Sort by rating in descending order
    user_rated_restaurants = user_ratings.sort_values(by='rating', ascending=False)
    
    return user_rated_restaurants

# Example usage: Show previously rated restaurants for user_id 'U0001'
user_id = 'U0001'
previous_ratings = get_previous_ratings(user_id, usersorder_df)

# Display results
print(f"Previously rated restaurants by user {user_id}:")
for _, row in previous_ratings.iterrows():
    print(f"Restaurant name: {row['Name']}, Rating: {row['rating']},Price:{row['cost']},Cuisines:{row['Cuisines']}")


Previously rated restaurants by user U0001:
Restaurant name: Burger Seigneur, Rating: 5,Price:700,Cuisines:Cafe, Burger, Continental, Beverages, Desserts
Restaurant name: Bangla Aahar, Rating: 4,Price:800,Cuisines:Bengali, Street Food
Restaurant name: BIB - Breakfast In The Box, Rating: 2,Price:400,Cuisines:Continental
Restaurant name: Waffle Dreams, Rating: 1,Price:400,Cuisines:Desserts, Fast Food, Beverages, Shake
Restaurant name: The Cheezy Oven, Rating: 1,Price:900,Cuisines:Pizza, Fast Food, Ice Cream, Beverages
Restaurant name: Waffle Head, Rating: 1,Price:900,Cuisines:Desserts, Waffle


In [4]:
# Create a user-item matrix (rows: users, columns: restaurants, values: ratings)
user_item_matrix = usersorder_df.pivot_table(index='user_id', columns='Name', values='rating')

# Fill NaN values with 0 (or use a different imputation strategy)
user_item_matrix = user_item_matrix.fillna(0)

# Compute the similarity between users (cosine similarity)
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Recommend restaurants for a given user based on collaborative filtering
def recommend_restaurants(user_id, user_item_matrix, user_similarity_df, top_n=5):
    # Find similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]  # Exclude the user itself
    
    # Get the ratings of similar users
    similar_users_ratings = user_item_matrix.loc[similar_users]
    
    # Weighted average of ratings (based on user similarity)
    weighted_ratings = similar_users_ratings.T.dot(user_similarity_df[user_id].loc[similar_users])
    weighted_ratings = weighted_ratings / user_similarity_df[user_id].loc[similar_users].sum()
    
    # Get restaurants the target user has not rated
    user_rated_restaurants = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index
    recommendations = weighted_ratings.drop(user_rated_restaurants)
    
    # Return the top N recommendations
    return recommendations.sort_values(ascending=False).head(top_n)


In [5]:
import pandas as pd

# Assuming 'usersorder_df' is the DataFrame that contains all the restaurant details
# Example of the structure of usersorder_df:
# usersorder_df = pd.DataFrame({
#     'rest_id': [...],
#     'Name': [...],
#     'user_id': [...],
#     'cost': [...],
#     'rating': [...],
#     'location': [...],
#     'Cuisines': [...]
# })

user_id = 'U0001'
recommendations = recommend_restaurants(user_id, user_item_matrix, user_similarity_df, top_n=5)

print(f"Top 5 recommended restaurants for user {user_id}:")

# Loop over the recommendations and fetch the details
for restaurant_name, score in recommendations.items():
    # Get the details of the restaurant from usersorder_df based on the restaurant name
    restaurant_details = usersorder_df[usersorder_df['Name'] == restaurant_name].iloc[0]
    
    # Extract details from the row
    cost = restaurant_details['cost']
    rating = restaurant_details['rating']
    cuisines = restaurant_details['Cuisines']
    
    # Print the details of the recommended restaurant
    print(f"Restaurant name: {restaurant_name}, Cost: {cost}, Cuisines: {cuisines}")


Top 5 recommended restaurants for user U0001:
Restaurant name: Sri Krishna Delicacy, Cost: 300, Cuisines: South Indian, North Indian
Restaurant name: Donne Biryani Grand, Cost: 200, Cuisines: Biryani, South Indian
Restaurant name: SLV Delite, Cost: 300, Cuisines: South Indian, North Indian
Restaurant name: Gourmet Ice Cream Cakes By Baskin Robbins, Cost: 200, Cuisines: Bakery, Desserts, Ice Cream
Restaurant name: Kolkata Famous Kati Rolls, Cost: 600, Cuisines: Rolls
